In [165]:
# s='PLEASANTLY'
# t='MEANLY'
# answer: 5

In [1]:
from Bio import SeqIO
import math

seq_name, seq_string = [], []
for seq_record in SeqIO.parse(open("rosalind_editband.txt", "r"),'fasta'):
    seq_name.append(seq_record.name)
    seq_string.append(str(seq_record.seq))
s, t = seq_string

In [2]:
class MinHeap:
    def __init__(self):
        self.heap = []
        self.size = 0
        self.index = {} #[math.inf] * num_of_nodes
    
    def parent(self, i):
        return (i-1)//2
    
    def left_child(self, i):
        return 2*i + 1
    
    def right_child(self, i):
        return 2*i + 2
    
    def find_min(self):
        if not self.heap:
            return None
        return self.heap[0]
    
    def is_empty(self):
        return not self.heap

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        self.index[self.heap[i][0]] = i
        self.index[self.heap[j][0]] = j
    
    def insert(self, node, value):
        #if self.index[node] != math.inf:
        if node in self.index:
            self.update(node, value)
        else:
            self.heap.append((node, value))
            self.size += 1
            self.index[node] = self.heap.index((node,value))
            self.heapify_up(self.size-1)

    def update(self, node, value):
        i = self.index[node]
        self.heap[i] = (node, value)
        self.heapify_up(i)
        self.heapify_down(i)
        # self.index[node] = self.heap.index((node,value)) ???

    def heapify_up(self, i):
        while i != 0 and self.heap[self.parent(i)][1] > self.heap[i][1]:
            self.swap(i, self.parent(i))
            i = self.parent(i)
    
    def pop_min(self):
        if self.size == 0:
            raise IndexError("Heap is empty")
        node, value = self.heap[0]
        self.heap[0] = self.heap[self.size-1]
        last_node, last_value = self.heap.pop()
        self.size -= 1
        del self.index[node]
        # self.index[last_node] = math.inf ??
        self.heapify_down(0)
        return node, value

    def heapify_down(self, i):
        while (self.left_child(i) < self.size and 
               self.heap[i][1] > self.heap[self.left_child(i)][1]) or \
              (self.right_child(i) < self.size and 
               self.heap[i][1] > self.heap[self.right_child(i)][1]):
            min_child = self.left_child(i)
            if (self.right_child(i) < self.size and 
                self.heap[self.right_child(i)][1] < self.heap[min_child][1]):
                min_child = self.right_child(i)
            self.swap(i, min_child)
            i = min_child
    
    def __len__(self):
        return self.size
    
    def __contains__(self, node):
        return any(node == item[0] for item in self.heap)
    
    def __getitem__(self, node):
        for item in self.heap:
            if node == item[0]:
                return item[1]
        raise KeyError(f"Node {node} not in heap")
    
    def __iter__(self):
        return iter(sorted(self.heap, key=lambda x: x[1]))


In [3]:
# this class represents a directed graph using
# adjacency list representation
class Graph:
	def __init__(self): 
		self.adj = {}

	def addEdge(self, u: int, v: int, w: int):
		if u in self.adj:
			self.adj[u].append((v, w))
		else:
			self.adj[u] = [(v, w)]

In [4]:
def dijkstra(src, sink, s, t):
    """ 
    use dijkstra's algo to get shortest distance from src to sink vertices of graph g
    construct the edit graph on the fly

    dist[v] - shortest distance from vertex src to vertex v (inf if not known)
    """

    #initiate the edit graph
    g = Graph()

    # priority queue to store vertices that
    # are being preprocessed
    pq = MinHeap()
    pq.insert(src, 0)

    # initialize dist
    dist = {} #dist = [math.inf] * g.V
    for i in range(0, len(s)+1):
        for j in range(0, len(t)+1):
            dist[(i,j)] = math.inf
    dist[src] = 0

    # while pq not empty
    while pq: 
        # from pq extract u, s.t distance from srt to u is min
        u, d = pq.pop_min()
        #check if popped sink node then return its' distance from src
        if u == sink:
            return d

        # last row or last column nodes do not have neighbors
        if u[0] == len(s) or u[1] == len(t):
            continue

        #add neighbors to the graph
        g.addEdge((u[0],u[1]), (u[0]+1,u[1]), 1) #u[0] is i, u[1] is j
        g.addEdge((u[0],u[1]), (u[0],u[1]+1), 1)
        g.addEdge((u[0],u[1]), (u[0]+1,u[1]+1), (1 if s[u[0]] != t[u[1]] else 0))

        # for neighbors of u
        for v, weight in g.adj[u]:
            # if there is shorter path to v through u.
            if dist[v] > dist[u] + weight:
                # update distance of v
                dist[v] = dist[u] + weight
                pq.insert(v, dist[v])

    return dist[sink]


In [5]:
dijkstra(src=(0,0), sink=(len(s), len(t)), s=s, t=t)

324

In [8]:
dijkstra(s,t,0,1,1)